<a href="https://colab.research.google.com/github/KorvenDalas/CMF/blob/main/CMF_CW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

# Importing data from Yahoo Finance
import yfinance as yf

# Import datetime to handle date and time operations
import datetime

#Option Pricing

## Designing

In [34]:
# Define the ticker for BAE Systems (BA.L)
ticker = 'BA.L'

# Define the time period
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=2 * 365)  # Approximately two years from now

# Download the data for BAE Systems
data = yf.download(ticker, start=start_date, end=end_date)

# Display the last few rows of the data
print(data.tail())

[*********************100%***********************]  1 of 1 completed

Price      Adj Close   Close    High          Low    Open   Volume
Ticker          BA.L    BA.L    BA.L         BA.L    BA.L     BA.L
Date                                                              
2024-12-05    1247.0  1247.0  1256.0  1237.000000  1249.5  3979387
2024-12-06    1229.5  1229.5  1251.0  1228.500000  1244.0  4419294
2024-12-09    1199.0  1199.0  1235.5  1190.150024  1234.0  7141046
2024-12-10    1179.0  1179.0  1195.0  1173.500000  1189.0  8578212
2024-12-11    1198.0  1198.0  1199.5  1177.000000  1177.5  6157147


In [35]:
# Calculate daily log returns
data['Log Return'] = np.log(data['Close'] / data['Close'].shift(1))

# Drop the first NaN value
data.dropna(inplace=True)

# Calculate daily volatility (standard deviation of log returns)
daily_volatility = data['Log Return'].std()

# Annualize the volatility
annualized_volatility = daily_volatility * np.sqrt(252)

print(f"Annualized Volatility (σ): {annualized_volatility:.4f}")

Annualized Volatility (σ): 0.2074


## BSM & Monte Carlo Simulation

In [36]:
# Parameters designed option
S0 = 12.295 # Stock price 06-12-2024
K = 12.5   # Strike price
sigma = 0.2070  # Volatility
T = 0.5 # Time to maturity in years
r = 0.0428  # Risk-free rate

np.random.seed(2024)  # For reproducibility
n_simulations = 50000  # Number of Monte Carlo simulations

In [37]:
from scipy.stats import norm
import scipy.stats as si

# Black-Scholes-Merton Formula
def euro_option_bs(S, K, T, r, sigma, payoff):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if payoff == "call":
        option_value = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        option_value = - S * si.norm.cdf(-d1, 0.0, 1.0) + K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)

    return option_value


# Monte Carlo Simulation
def monte_carlo_option(S0, K, T, r, sigma, n_simulations):
    Z = np.random.standard_normal(n_simulations)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    payoffs = np.maximum(ST - K, 0)
    mc_price = np.exp(-r * T) * np.mean(payoffs)
    return mc_price


In [38]:
# Calculate Prices
bs_price = euro_option_bs(S0, K, T, r, sigma, 'call')
mc_price = monte_carlo_option(S0, K, T, r, sigma, n_simulations)

# Display results
print(f"Option Price using Black-Scholes Model: {bs_price:>15.5f}")
print(f"Option Price using Monte Carlo Simulation: {mc_price:>12.5f}")

Option Price using Black-Scholes Model:         0.74579
Option Price using Monte Carlo Simulation:      0.75618


# Implied Volatility

## Newton-Raphson Method Call

In [39]:
def newton_vol_call(S, K, T, C, r):

    #S: spot price
    #K: strike price
    #T: time to maturity
    #C: Call value
    #r: risk free rate
    #sigma: volatility of underlying asset

    MAX_ITERATIONS = 10000
    tolerance = 0.000001

    sigma = 0.25

    for i in range(0, MAX_ITERATIONS):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        price = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
        vega = S * np.sqrt(T) * si.norm.pdf(d1, 0.0, 1.0)

        diff = C - price

        if (abs(diff) < tolerance):
            return sigma

        # Updated sigma with safeguard for small Vega
        if vega < 1e-10:
            raise ValueError("Vega is too small, iteration may not converge.")

        sigma = sigma + diff/vega

    # If not converged, return last estimate
    raise ValueError("Newton-Raphson did not converge within the maximum iterations.")

In [40]:
def newton_vol_put(S, K, T, P, r):

    #S: spot price
    #K: strike price
    #T: time to maturity
    #P: Put value
    #r: risk free rate
    #sigma: volatility of underlying asset

    MAX_ITERATIONS = 10000
    tolerance = 0.000001

    sigma = 0.25

    for i in range(0, MAX_ITERATIONS):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        price = K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)-S * si.norm.cdf(-d1, 0.0, 1.0)
        vega = S * np.sqrt(T) * si.norm.pdf(d1, 0.0, 1.0)

        diff = P - price

        if (abs(diff) < tolerance):
            return sigma

        # Updated sigma with safeguard for small Vega
        if vega < 1e-10:
            raise ValueError("Vega is too small, iteration may not converge.")

        sigma = sigma + diff/vega

    # If not converged, return last estimate
    raise ValueError("Newton-Raphson did not converge within the maximum iterations.")

## Tesla Implied Volatility

In [41]:
from datetime import datetime

def time_to_maturity(expiration_date: str) -> float:
    # Parse the expiration date
    expiry = datetime.strptime(expiration_date, '%Y-%m-%d')

    # Get the current date
    today = datetime.now()

    # Calculate the time difference in days
    days_to_expiry = (expiry - today).days

    # Convert days to years (365 days in a year)
    return days_to_expiry / 365.0

In [42]:
def get_spot_price(ticker: str) -> float:
    # Fetch the stock data
    stock = yf.Ticker(ticker)

    # Get the current spot price
    spot_price = stock.history(period = "1d")["Close"].iloc[-1]

    return spot_price

In [43]:
def get_call_value(ticker: str, expiration_date: str, strike_price: float) -> float:

    stock = yf.Ticker(ticker)
    option_chain = stock.option_chain(date = expiration_date)
    calls = option_chain.calls

    # Filter for the specific strike price
    call_row = calls[calls["strike"] == strike_price]

    if not call_row.empty:
        call_value = call_row["lastPrice"].iloc[0]
    else:
        call_value = None  # If the strike price is not found

    return call_value

In [44]:
def get_put_price(ticker: str, expiration_date: str, strike_price: float) -> float:
    stock = yf.Ticker(ticker)
    option_chain = stock.option_chain(date = expiration_date)
    puts = option_chain.puts

    # Filter for the specific strike price
    put_row = puts[puts["strike"] == strike_price]

    if not put_row.empty:
        put_price = put_row["lastPrice"].iloc[0]
    else:
        put_price = None  # If the strike price is not found

    return put_price

### Tesla Put option, K = 140

In [45]:
# Define the parameters, K = 140
ticker = "TSLA"
expiration_date = "2025-01-17"
strike_price = 140
risk_free_rate = 0.05

S = get_spot_price(ticker)
K = strike_price
T = time_to_maturity(expiration_date)
P = get_put_price(ticker, expiration_date, strike_price)
r = risk_free_rate

print(f"Spot Price (S): ${S:.2f}")
print(f"Put Price (P) for Strike ${K}: ${P:.2f}")

sigma = newton_vol_put(S, K, T, P, r)

print(f"Implied Volatility (σ): {sigma:.4f}")

Spot Price (S): $419.25
Put Price (P) for Strike $140: $0.16


ValueError: Vega is too small, iteration may not converge.

### Tesla Put option, K = 340

In [46]:
# Define the parameters, K = 340
ticker = "TSLA"
expiration_date = "2025-01-17"
strike_price = 340
risk_free_rate = 0.05

S = get_spot_price(ticker)
K = strike_price
T = time_to_maturity(expiration_date)
P = get_put_price(ticker, expiration_date, strike_price)
r = risk_free_rate

print(f"Spot Price (S): ${S:.2f}")
print(f"Put Price (P) for Strike ${K}: ${P:.2f}")

sigma = newton_vol_put(S, K, T, P, r)

print(f"Implied Volatility (σ): {sigma:.4f}")

Spot Price (S): $419.10
Put Price (P) for Strike $340: $5.12


ValueError: Vega is too small, iteration may not converge.

## Tesla Historical Volatility

In [47]:
TSLA = yf.download("TSLA", start="2023-12-10", end="2024-12-10")

[*********************100%***********************]  1 of 1 completed


In [48]:
S = TSLA['Adj Close'].iloc[-1]
spot_price = S.values[0]
print(f"The spot price is ${round(spot_price, 2)}")

The spot price is $389.79


In [49]:
log_return = np.log(TSLA['Adj Close'] / TSLA['Adj Close'].shift(1))
vol_h = np.sqrt(252) * log_return.std()
historical_volatility = vol_h.values[0]
print(f"The historical volatility is {round(historical_volatility * 100,2)} %")

The historical volatility is 60.87 %


# Binomial Tree

In [50]:
# Given parameters
S0 = 100        # Initial stock price
sigma = 0.20    # Volatility
r = 0.05        # Risk-free rate
delta_t = 1/3   # Time step (4 months in a year)

# Calculating u, d, and a
u = math.exp(sigma * math.sqrt(delta_t))    # Up factor
d = math.exp(-sigma * math.sqrt(delta_t))   # Down factor
a = math.exp(r * delta_t)                   # Risk-free growth factor

# Calculating risk-neutral probability p
p = (a - d) / (u - d)

# Print results
print(f"u : {u:.3f} (Up factor)")
print(f"d : {d:.3f} (Down factor)")
print(f"a : {a:.3f} (Risk-free growth factor)")
print(f"p : {p:.3f} (Risk-neutral probability)")

u : 1.122 (Up factor)
d : 0.891 (Down factor)
a : 1.017 (Risk-free growth factor)
p : 0.544 (Risk-neutral probability)


## American call option, K = $100, three-step tree

In [51]:
import os

# Parameters
K = 100.0               # Strike price
N = 3                   # Number of time steps
payoff = "call"         # Option type

# Initialize price tree
S = np.zeros((N + 1, N + 1))
S[0, 0] = S0            # Initial stock price
z = 1                   # Tracks states per time step

# Build binomial tree
for t in range(1, N + 1):                   # Loop over time steps
    for i in range(z):                      # Loop over states
        S[i, t] = S[i, t - 1] * u           # Up move
        S[i + 1, t] = S[i, t - 1] * d       # Down move
    z += 1                                  # Increment states

S  # Resulting price tree

array([[100.        , 112.24009024, 125.97837858, 141.39824581],
       [  0.        ,  89.09472523, 100.        , 112.24009024],
       [  0.        ,   0.        ,  79.37870064,  89.09472523],
       [  0.        ,   0.        ,   0.        ,  70.72223522]])

In [52]:
S_T = S[:, -1]                              # Extract final stock prices from the tree
V = np.zeros((N + 1, N + 1))                # Initialize value tree for option pricing

# Calculate option payoff at maturity
if payoff == "call":
    V[:, -1] = np.maximum(S_T - K, 0.0)     # Payoff for call option
elif payoff == "put":
    V[:, -1] = np.maximum(K - S_T, 0.0)     # Payoff for put option

V  # Resulting option value tree

array([[ 0.        ,  0.        ,  0.        , 41.39824581],
       [ 0.        ,  0.        ,  0.        , 12.24009024],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [53]:
# For American options
if payoff == "call":
    for j in range(N - 1, -1, -1):          # Loop backward through time steps
        for i in range(j + 1):              # Loop through states at each step
                                            # Max of early exercise or theoretical value
            V[i, j] = np.maximum(S[i, j] - K, (p * V[i, j + 1] + (1 - p) * V[i + 1, j + 1]) / a)

elif payoff == "put":
    for j in range(N - 1, -1, -1):          # Loop backward through time steps
        for i in range(j + 1):              # Loop through states at each step
                                            # Max of early exercise or theoretical value
            V[i, j] = np.maximum(K - S[i, j], (p * V[i, j + 1] + (1 - p) * V[i + 1, j + 1]) / a)

V  # Final option value tree


array([[11.04387109, 17.71388824, 27.6312332 , 41.39824581],
       [ 0.        ,  3.50065379,  6.54586268, 12.24009024],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])